In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

from tqdm import tqdm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv('병합_selected_features.csv')
df

,ID,Segment,이용금액_오프라인_R3M,이용금액대_ord,정상청구원금_B2M,이용금액_R3M_신용체크,연속유실적개월수_기본_24M_카드,미이용_CA,정상청구원금_B0M,최대이용금액_체크_R12M,...,RP후경과월_전기,이용금액_카드론_R12M,이용건수_체크_R6M,RV_평균잔액_R12M,이용개월수_할부_유이자_R6M,_3순위쇼핑업종_이용금액,잔액_할부_유이자_B0M,잔액_현금서비스_B2M,이용가능여부_해외겸용_본인,RV_최대잔액_R12M
0,TRAIN_000000,D,11756,6,15251,-454,17,0,15067,998,...,6,0,0,0,0,0,0,30640,0,0
1,TRAIN_000001,E,12128,5,2776,7089,17,0,2222,0,...,6,0,0,0,0,517,1491,0,0,0
2,TRAIN_000002,C,24370,6,23325,27336,8,0,26184,0,...,6,0,0,1255,0,1172,0,27654,1,4587
3,TRAIN_000003,D,12529,6,18808,4270,24,0,20959,0,...,6,0,0,0,4,682,2092,28938,1,0
4,TRAIN_000004,E,0,2,0,9385,0,1,639,3910,...,6,0,55,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,TRAIN_399995,E,0,1,0,10755,0,1,19,5072,...,6,0,76,0,0,0,0,0,1,0
399996,TRAIN_399996,D,25205,6,14844,27636,17,1,13462,0,...,6,0,0,0,0,346,0,0,1,0
399997,TRAIN_399997,C,13267,3,6862,23187,24,1,7049,0,...,6,0,0,0,0,305,0,0,1,0
399998,TRAIN_399998,E,0,1,0,0,0,1,0,0,...,6,0,0,0,0,0,0,0,0,0


In [3]:
# ID와 Segment를 먼저 제거하고, 숫자형 컬럼만 선택합니다
df_vif = (
    df
    .drop(columns=['ID', 'Segment'])
    .select_dtypes(include=[np.number])
    .copy()
)

threshold = 10
removed_features = []

In [4]:
while True:
    # 상수항을 추가하고 결측값을 0으로 채기기
    X = sm.add_constant(df_vif).fillna(0)
    
    # tqdm으로 진행상황을 표시하며 VIF 계산
    vif = [variance_inflation_factor(X.values, i) for i in tqdm(range(X.shape[1]), desc="VIF 계산중")]
    print('vif done')
    vif_df = pd.DataFrame({'feature': X.columns, 'vif': vif})
    display(vif_df)
    
    # 상수항은 제외
    vif_df = vif_df[vif_df['feature'] != 'const']
    
    # 기준을 넘는 변수가 있으면 제거하고 리스트에 추가
    high_vif_features = vif_df[vif_df['vif'] > threshold]['feature'].tolist()
    if not high_vif_features:
        break
    removed_features.extend(high_vif_features)
    df_vif.drop(columns=high_vif_features, inplace=True)

VIF 계산중: 100%|████████████████████████████████████████████████████████████████████| 166/166 [06:29<00:00,  2.35s/it]

vif done


,feature,vif
0,const,0.000000e+00
1,이용금액_오프라인_R3M,1.257073e+02
2,이용금액대_ord,3.834962e+00
3,정상청구원금_B2M,2.638529e+01
4,이용금액_R3M_신용체크,1.201557e+10
...,...,...
161,_3순위쇼핑업종_이용금액,1.099788e+01
162,잔액_할부_유이자_B0M,3.651714e+00
163,잔액_현금서비스_B2M,2.402307e+01
164,이용가능여부_해외겸용_본인,1.653277e+01


VIF 계산중: 100%|██████████████████████████████████████████████████████████████████████| 67/67 [00:54<00:00,  1.22it/s]

vif done


,feature,vif
0,const,742.300845
1,이용금액대_ord,3.320279
2,연속유실적개월수_기본_24M_카드,2.824312
3,미이용_CA,3.799319
4,미이용_할부,1.888485
...,...,...
62,RV일시불이자율_할인전,2.472474
63,RP후경과월_전기,1.009384
64,이용금액_카드론_R12M,1.213326
65,이용개월수_할부_유이자_R6M,1.921038


In [5]:
# 제거된 변수 목록과 최종 VIF 결과를 출력
print("제거된 변수들:", removed_features)
print(vif_df.reset_index(drop=True))

제거된 변수들: ['이용금액_오프라인_R3M', '정상청구원금_B2M', '이용금액_R3M_신용체크', '정상청구원금_B0M', '최대이용금액_체크_R12M', '최대이용금액_CA_R12M', '이용건수_신용_R12M', '이용금액_오프라인_B0M', '입회일자_신용', '이용금액_체크_R12M', '이용금액_체크_R3M', '이용금액_체크_R6M', '입회경과개월수_신용', '이용금액_일시불_R12M', '이용건수_CA_R12M', '이용개월수_신용_R12M', '이용건수_오프라인_R6M', '평잔_카드론_3M', '카드이용한도금액_B2M', '이용개월수_신용_R6M', '이용금액_CA_R6M', '최대이용금액_할부_무이자_R12M', '증감율_이용금액_일시불_분기', '이용건수_일시불_B0M', '카드이용한도금액_B1M', '이용금액_CA_R12M', '할부금액_6M_R12M', '이용개월수_CA_R3M', '이용개월수_체크_R12M', '한도증액횟수_R12M', '이용금액_오프라인_R6M', '평잔_카드론_6M', '이용건수_신판_B0M', '최대이용금액_할부_R12M', '이용개월수_CA_R12M', '이용금액_할부_무이자_R12M', '이용개월수_CA_R6M', '수신거부여부_메일', '이용금액_할부_R12M', '소지카드수_이용가능_신용', 'CA이자율_할인전', '이용후경과월_신판', '증감율_이용금액_신판_분기', '이용건수_신판_R12M', '_1순위카드이용건수', '이용가맹점수', '이용개월수_일시불_R12M', '이용금액_체크_B0M', '카드이용한도금액', '이용후경과월_신용', '유효카드수_신용체크', '이용건수_오프라인_R3M', '이용건수_일시불_R3M', '이용건수_일시불_R6M', '정상청구원금_B5M', '이용개월수_체크_R6M', '증감율_이용금액_신용_분기', '이용개월수_체크_R3M', '이용건수_신판_R6M', '이용건수_체크_R3M', '유효카드수_신용', '이용금액_일시불_R6M', '잔액_카드론_B5M', '경과일수

In [7]:
1/0

ZeroDivisionError: division by zero

In [9]:
# 최종 선택된 피처 목록 생성
selected_features = df_vif.columns.tolist()

# 결과 데이터프레임에 ID, Segment와 선택된 피처 병합
result_df = df[['ID', 'Segment'] + selected_features]

# 선택된 피처 저장
result_df.to_csv('selected_features.csv', index=False, encoding='utf-8-sig')

# 제거된 피처 저장
result_df = df[removed_features].copy()
result_df.to_csv('removed_features.csv', index=False, encoding='utf-8-sig')

In [12]:
# 테스트 데이터 불러오기
test_df = pd.read_csv("merged_2018_test.csv")  # 파일명은 실제 파일명으로 바꿔줘

# 보존할 기본 컬럼 지정
keep_cols = ['ID', 'Segment']  # 혹시 Segment가 없으면 'ID'만 남기면 됨

# 실제 존재하는 selected_features만 필터링
available_features = [col for col in selected_features if col in test_df.columns]

# 유지할 전체 컬럼 목록 구성
final_cols = [col for col in keep_cols if col in test_df.columns] + available_features

# 해당 컬럼들만 추출
filtered_test_df = test_df[final_cols].copy()

# 결과 저장
filtered_test_df.to_csv("test_selected_features.csv", index=False, encoding='utf-8-sig')

In [13]:
# 제거된 피처 목록 불러오기
removed_df = pd.read_csv("removed_features.csv")
removed_features = removed_df.columns.tolist()  # 컬럼명만 추출

# test 전체 데이터 불러오기 
test_full_df = pd.read_csv("merged_2018_test.csv") 

# 3. 제거된 피처만 추출
test_removed_df = test_full_df[removed_features].copy()
test_removed_df.to_csv("test_removed_features.csv", index=False, encoding='utf-8-sig')